# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [3]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [309]:
# Функция обработки неправильных разбиений без эвристики (см. ниже функцию __fit_threshold)
def simple_split(x, x_l, x_r):
    return (x_l + x_r) / 2.0

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None,
                 split_function=None):
        self.tree = dict()
        # min_samples_split - минимальное количество значений после разделения
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise
            
        # Функция эврестической обработки разбиения выборки, когда по бокам одинаковые значения x
        if split_function is None:
            self.split_function = simple_split
        else:
            self.split_function = split_function
    
    # Делает массив долей из массива частот в делении
    def __get_parts(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_c/l_s, r_c/r_s
    
    # Возвращает взвешенную неопределенность (|S_l|*Imp(S_l)+|S_r|*Imp(S_r)).
    # Деление на |S| не нужно (=const в узле)
    def __weight_impurity(self, l_s, impurity_l, l_r, impurity_r):
        impurity_l = impurity_l.reshape(-1, 1)
        impurity_r = impurity_r.reshape(-1, 1)
        return l_s*impurity_l+l_r*impurity_r
    
    # l_c - количество элементов каждого класса слева
    # l_s - размер левого класса
    # Аналогично для r_c, r_s
    def __gini(self, l_c, l_s, r_c, r_s):
        l_p, r_p = self.__get_parts(l_c, l_s, r_c, r_s)
        return self.__weight_impurity(l_s, 1-np.sum(l_p*l_p, axis=1), r_s, 1-np.sum(r_p*r_p, axis=1)) # Ваш код
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_p, r_p = self.__get_parts(l_c, l_s, r_c, r_s)
        return self.__weight_impurity(l_s, -np.sum(l_p*np.log(l_p), axis=1),
                                      r_s, -np.sum(r_p*np.log(r_p), axis=1)) # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_p, r_p = self.__get_parts(l_c, l_s, r_c, r_s)
        return self.__weight_impurity(l_s, 1-np.max(l_p, axis=1), r_s, 1-np.max(r_p, axis=1)) # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[0:int(np.sqrt(n_feature))]# Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[0:int(np.log2(n_feature))]# Ваш код

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)# Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        #print x[:, feature_id], threshold
        left_mask = x[:, feature_id] > threshold
        #print left_mask
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    # x - значения количественного признака у объектов на входе
    # y - соответствующие классы объектов на входе
    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортирует полученные объекты по значению признака x
        # Массив sorted_x - отсортированные значения признака, sorted_y - соответствующие классы
        # class_number - количество различных классов среди полученных объектов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Рассчитывает индексы пороговых значений (где меняется класс объекта), которые будут рассматриваться в дереве
        # Индексы - начала новых классов
        # Первая строчка - гарантия того, что после любого разбиения в каждом классе будет
        # min_samples_split значений (индексы разбиений не будут меньше self.min_samples_split + 1) - "обрезание"
        # Вторая строчка - расчет индексов границ. splitted_sorted_y[:-1] != splitted_sorted_y[1:] - 
        # даст True, если предыдущее значение в массиве не совпадает со следующим
        # where(...)[0] - дает номера этих индексов, +(self.min_samples_split + 1) - потому что было смещение
        # в первой строчке
        # Добавлена эвристика. Рассмотрим выборку
        # sorted_x: 0 0 0 1 1 1
        # sorted_y: 1 1 0 0 1 1
        # Тогда при пороговых значениях 0 и 1 выборка якобы будет разделена на (1 1 ; 0 0 1 1) и (1 1 0 0 ; 1 1)
        # (якобы - потому что в данном алгоритме деление привязано к порогам, а не к значениям), но это неправильно
        # sorted_x: 6 6 7 7 8 8
        # sorted_y: 1 1 1 0 0 0
        # 7 - более-менее разумный threshold. Разные варианты условий с комментариями ниже.
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # Не могут быть слева и справа одновременно минимальный или максимальный x - это не разделит выборку
        #print "SortedX", sorted_x, "max", x.max(), "min", x.min()
        #r_border_ids = r_border_ids[((sorted_x[r_border_ids-1] != x.min()) | (sorted_x[r_border_ids] != x.min())) &
        #                            ((sorted_x[r_border_ids-1] != x.max()) | (sorted_x[r_border_ids] != x.max()))]
        # Получилось плохо - очень глубокое дерево за счет того, что теряется порог 0.5 (например, в примере выше)
        
        #print "RBI", r_border_ids
        #print "SortedY", sorted_y
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # Определяет количество объектов каждого типа в каждом классе после деления, объекты "обрезанные" слева
        # включаются в первый класс (с нулевым индексом)
        # eq_el_count - количество объектов, находящихся между 2мя соседними рассматриваемыми границами
        # без учета "боковых" объектов (гарантировавших min_samples_split)
        # one_hot_code - указывает, какой класс был предыдущим на этой границе в виде one-hot encoding
        # class_increments - таблица: границы\классы с указанием количества объектов между границами (инициализация)
        # class_increments - таблица: количество объектов каждого типа входит в классах (разделенных по границам)
        # "Обрезанные" значения в начале включаются в первый класс
        # Ошибка - в последней строчке sorted_y - исправлено
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Рассчитывает параметры разбиений: количество объектов каждого класса в частях разбиения
        # l_class_count - количество объектов каждого типа, остающихся слева при разбиении по каждой границе
        # l_class_count - 2D массив (cout_borders, count_classes)
        # r_class_count - аналогичный 2D массив, количество объектов, остающихся справа при разбиении по каждой границе
        # l_sizes - размеры левых классов при разбиении по каждой границе - столбец
        # r_sizes - аналогично для правых классов
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Выбирает оптимальное разбиение (соответствующее минимуму взвешенной меры неопределенности конечных классов -
        # максимуму прироста информации). Взвешенная неопределенность берется без деления на |S| = const
        # gs - мера неопределенности для всех разбиений
        # idx - номер оптимального
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #print gs.shape
        idx = np.argmin(gs)
        #print "TH", (sorted_x[r_border_ids-1]+sorted_x[r_border_ids])/2.0
        #print "GS", gs, l_class_count, r_class_count
    
        # Что делает этот блок кода?
        # Возвращает оптимальное разбиение: взвешенную энтропию при оптимальном разбиении,
        # trashhold (среднее значение количественного признака до и после границы класса)
        left_el_id = l_sizes[idx][0]
        
        # Функция split_function (правила обработки threshhold) вынесена за пределы класса, чтобы ее можно было поменять
        # разбиения, когда по бокам одинаковые значения x
        return gs[idx], self.split_function(sorted_x, sorted_x[left_el_id-1], sorted_x[left_el_id])
    
    # Создает лист
    def __create_leaf(self, y):
        probabilities = np.bincount(y, minlength=self.num_class)/float(len(y))
        # В predict - node[1] - самый популярный класс
        # node[2] - вероятности
        return (self.LEAF_TYPE, probabilities.argmax(), probabilities)
    
    # Определяет, нужно создавать лист (тогда вернет 0) или вершину (тогда -1)
    def __what_create(self, y):
        max_part = np.bincount(y).max()/len(y)
        return -1 if (max_part < self.sufficient_share) else 0
    
    # Мы не различаем количественные и категориальные признаки
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Что такое pred_f? Что такое sufficient_share - максимальная доля?
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE +
        # self.NON_LEAF_TYPE +

        # self.tree +
        # self.max_depth +
        # self.sufficient_share (?)
        # self.min_samples_split +

        # self.get_feature_ids +
        # self.__find_threshold +
        # self.__div_samples +
        # self.__fit_node +
        
        # Пока будем считать pred_f = 0 => это нужно сделать листом
        #print "start", pred_f, len(y)
        # Пустой узел не создается
        if (len(y) == 0):
            return
        if (pred_f == 0) or (len(y) <= self.min_samples_split*2):
            self.tree[node_id] = self.__create_leaf(y)
            return
        if (depth == self.max_depth) and not (depth is None):
            self.tree[node_id] = self.__create_leaf(y)
            return
        
        features_for_node = self.get_feature_ids(x.shape[1])
        possible_conditions = list() #node - tuple: (feature_id, threshold)
        impurities = list()
        for feature in features_for_node:
            #print "Feature", feature
            impurity, threshold = self.__find_threshold(x[:, feature], y)
            #print "I - T", impurity, threshold
            possible_conditions.append((feature, threshold))
            impurities.append(impurity)
        
        result_feature, result_threshold = possible_conditions[np.array(impurities).argmin()]
        #print "RF, RT", result_feature, result_threshold
        splitted_observations = self.__div_samples(x, y, result_feature, result_threshold)
        #print splitted_observations
        
        # Если при разбиении образовался пустой класс, то мы работаем с листом
        if (len(splitted_observations[0]) == 0) or (len(splitted_observations[1]) == 0):
            self.tree[node_id] = self.__create_leaf(y)
            return
        #print "EMPTY", has_empty_class
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, result_feature, result_threshold)
        self.__fit_node(splitted_observations[0], splitted_observations[2], node_id*2 + 1, depth+1,
                        self.__what_create(splitted_observations[2]))
        self.__fit_node(splitted_observations[1], splitted_observations[3], node_id*2 + 2, depth+1,
                        self.__what_create(splitted_observations[3]))
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        #print x
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            #print node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [310]:
# Эвристика. Если sorted_x[left_el_id-1] = sorted_x[left_el_id] = max(x), 
# то, вероятно, "хорошее" разбиение отсекает максимальные значения x. Попробуем это указать (для мин - аналогично)
# Возвращет полусумму x1 и x2 в общем случае, (x1+pre_max_x)/2, если x1=x2=max, (x1+pre_min_x)/2, если x1=x2=min
# Для обработки предполагаемых разбиений, которые реально не разбивают группу
def evristic_split(x, x_l, x_r):
    # Получилось лучше, чем с фильтрацией массива r_border_ids, но все равно медленно - 5 сек.
    uniq_x = np.unique(x)
    if len(uniq_x) == 1:
        return (x_l + x_r) / 2.0
    if (x_l == uniq_x[0] and x_r == uniq_x[0]):
        return (uniq_x[0] + uniq_x[1]) / 2.0
    if (x_l == uniq_x[-1] and x_r == uniq_x[-1]):
        return (uniq_x[-1] + uniq_x[-2]) / 2.0
    return (x_l + x_r) / 2.0

In [311]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [312]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [313]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf_evr = MyDecisionTreeClassifier(min_samples_split=2, split_function=evristic_split)
clf = DecisionTreeClassifier(min_samples_split=2)

In [314]:
print y
my_clf.fit(x, y)

[1 0 0 ..., 0 0 0]


## Проверка скорости работы

In [315]:
print "Без эвристики"
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)
print len(my_clf.tree)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

Без эвристики
0.422000169754
351
0.952999830246


Таким образом, для алгоритма без эвристики задание по времени выполнено. Проверим точность.  

In [317]:
# Быстрее, чем sklearn - нужно отключить эврестическую обработку "неправильных" разрезаний выборки - см. выше
print "С эвристикой"
t1 = time()
my_clf_evr.fit(x, y)
t2 = time()
print(t2 - t1)
print len(my_clf_evr.tree)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

С эвристикой
4.6099998951
4957
0.969000101089


## Проверка качества работы

In [318]:
gkf = KFold(n_splits=5, shuffle=True)

In [319]:
print "Без эвристики"
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print "Len_tree", len(my_clf.tree)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

Без эвристики
Len_tree 533
0.931778498379
Len_tree 679
0.932152656523
Len_tree 2215
0.928369501954
Len_tree 2955
0.933108838447
Len_tree 3365
0.932815033468


In [320]:
print "C эвристикой"
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf_evr.fit(X_train, y_train)
    print "Len_tree", len(my_clf_evr.tree)
    print(accuracy_score(y_pred=my_clf_evr.predict(X_test), y_true=y_test))

C эвристикой
Len_tree 9063
0.926082979962
Len_tree 12355
0.925043651783
Len_tree 14847
0.923962750478
Len_tree 18787
0.9229234223
Len_tree 22267
0.924666361784


In [321]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.893032343893
0.893032343893
0.892533466367
0.889831213104
0.891115453374


Таким образом, на данном наборе данных алгоритм без эвристики работает лучше и удовлетворяет поставленным условиям.  
Однако, возможно, в другом случае будет лучше работать алгоритм с эвристикой.